<h1>Product page parser</h1>

<p>In this notebook we are going to parse a single product page.</p>
<p>According to the task we should get the following: </p>
   <ol>
  <li>categories</li>
  <li>subcategories</li> 
  <li>name</li>
  <li>price</li>
  <li>description</li>
  <li>charachteristics</li>
  <li>similar products</li>
  <li>product image link</li>
  <li>if product is a hit or not</li>
</ol> 

<h3><i>Important note</i></h3>
<p>Categories and subcategories (both level 1 and level 2) we plan to extract from parsing table obtained from the catalog main page. The parsing table currently contains 1222 rows. Each row has a link to products page with multiple product cards.</p>


In [5]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup
import pandas as pd
import re

<p>We will develop our parser on a bosch drill webpage which seems to be the most complete and contains all the features, including hit tag.</p>

In [4]:
url = 'https://www.oma.by/perforator-bosch-pbh-2100-re-2-zubila-2-bura-1-216760-p'
oma_main_html = requests.get(url)
product_page_soup = BSoup(oma_main_html.text, 'html.parser')